In [1]:

import pandas as pd
annotations_df = pd.read_csv("annotations.tsv", sep='\t') 
annotations_df.head()

,comment_id,abstract_id,annotator,score,comment
0,[1],1,"Dey, Gautam",conclusions_nounchange_1,NaN
1,[2],1,Jessica Polka,context_nounchange_1,NaN
2,[3],1,Mate Palfy,context_nounchange_1+,NaN
3,[4],1,jonathon coates,context_effect-_1,NaN
4,[5],1,Jessica Polka,conclusion_stat-_1,NaN


In [2]:
max_id = max([int(x) for x in set(annotations_df["abstract_id"])])
print (max_id)

184


In [3]:
annotations_df = annotations_df[annotations_df['score'].notna()]

dois = open("dois.txt","r").read().strip().split("\n")
dois = {x+1:dois[x] for x in range(len(dois))}

print (len(dois))
dois[184]

184


'10.1101/869313'

In [4]:
 normalise_sections = {'conclusion':"conclusions",
 'conclusion':"conclusions",
 'conclusion  ':"conclusions",
 'conclusions':"conclusions",
 'concusions':"conclusions",
 'contex':"context",
 'context':"context",
 'context  ':"context",
 'context  removed':"context",
 'countext':"context",
 'new':"new",
 'reslts':'results',
 'resuls':'results',
 'result':'results',
 'results':'results',
 'results/context':'results/context',
 'resutls':'results',
 'resuts':'results'}

normalise_labels = {'nounchange': 'nounchange', 'effect-': 'effect', 'stat-': 'stat', 'added': 'added', 'removed': 'removed', 'stat+': 'stat', 'stat-1': 'stat', 'effect+': 'effect', 'statinfo': 'stat', 'effectreverse': 'effectreverse', 'aded': 'added', 'effectt+': 'effect', 'nouchange': 'nounchange', 'effect': 'effect', 'results': 'results', 'nounchage': 'nounchange', 'stat': 'stat', 'ef': 'effect', 'added  ': 'added', 'emoved': 'removed', 'remove': 'removed'}

In [6]:
out_ann = open("raw_annotations.csv","w")
out_ann.write("page_id, doi,annotator,section,label,score\n")

final_columns = {}
for id in range(1,185):
    overall_score = {x:{p:[] for z,p in normalise_labels.items()} for y,x in normalise_sections.items()}
    scores = annotations_df.loc[annotations_df['abstract_id'] == id]["score"].tolist()
    annotators = annotations_df.loc[annotations_df['abstract_id'] == id]["annotator"].tolist()
    scores = [[annotators[x],scores[x]] for x in range(len(scores)) if "_" in scores[x]]
    doi = dois[id]
    for el in scores:
            ann = el[0].replace(","," ")
            score = el[1]
            score = score.replace(" ","")
            score = score.strip()

            if len(score.split("_"))>2 and score.split("_")[0] in normalise_sections:
                section = normalise_sections[score.split("_")[0]]
                label = normalise_labels[score.split("_")[1]]
                
                # check if there is a modifier in the label
                modifier = [x for x in score.split("_")[1] if x == "+" or x == "-"]
                sc = score.split("_")[-1].strip()
                try:
                    int_sc = int(''.join(c for c in sc if c.isdigit()))
                # this happens if the annotator forgot to assign a value, we add 1 here
                except ValueError:
                    sc = "1"
                    int_sc = 1

                if len(modifier)>0:
                    modifier = modifier[0]
                    # we check if there is a modifier in the score
                    sc_modifier = [x for x in sc if x == "+" or x == "-"]
                    if len(sc_modifier)>0 and modifier != sc_modifier[0]:
                        # so if the modifier in the label is inconsistent (it happens 5 times)
                        # for the moment I am always considering it negative
                        modifier = "-"

                # apply the modifier
                if modifier == "-":
                    int_sc = -int_sc

                out_ann.write(str(id)+","+doi+","+ann+","+section+","+label+","+str(int_sc)+"\n")
                overall_score[section][label].append(sc)


    for section,labels in overall_score.items():
        for label,score in labels.items():
            
            if section+"_"+label not in final_columns:
                final_columns[section+"_"+label] =[score]
            else:
                final_columns[section+"_"+label].append(score)
out_ann.close()

In [ ]:
output_columns = open("col.csv","w")

for column in final_columns.keys():
    output_columns.write(column+",")
output_columns.write("\n")

for i in range(186):
    for c,v in final_columns.items():
        s = v[i]
        s = " ".join([str(n) for n in s])
        output_columns.write(s+",")
    output_columns.write("\n")
output_columns.close()



In [ ]:
out = open("contains_2.txt","w")
for id in range(186):
    scores = annotations_df.loc[annotations_df['abstract_id'] == id]["score"]
    scores = [x.split("_")[-1].strip() for x in scores if "_" in x]
    #scores = [x for x in scores if "2" in x]
    scores = [x for x in scores if "2" in x]
    if len(scores)>0:
        out.write(str(len(scores))+"\n")
    else:
        out.write("0\n")
out.close()


In [ ]:
out = open("sum.txt","w")
for id in range(186):
    scores = annotations_df.loc[annotations_df['abstract_id'] == id]["score"]
    scores = [x.split("_")[-1].strip() for x in scores if "_" in x]
    #scores = " ".join([x for x in scores if "2" in x or "1" in x])
    scores = [x for x in scores if "-" not in x]
    scores = [int(y) for x in scores if "2" in x or "1" in x for y in x if y.isnumeric()]
    print (scores)
    if len(scores)>0:
        out.write(str(sum(scores))+"\n")
    else:
        out.write("0\n")
out.close()